In [3]:
# TODO: Sanity check with model
# TODO: Add comments and references
# using Pycall


function simulateBOLD(Z, dt, voxelCounts, X=None, F=None, Q=None, V=None)
    
    Nrow, Ncol = size(Z)
# #     TODO: Add the costumizable voxelcount but it's an array of number of voxels in each region
#             (not used yet in Py version either)
#     if "voxelCounts"∉ globals() find out where to find dictionary of "globals"
#         voxelCounts = ones(Nrow,Ncol)
#     end
    voxelCounts = ones(Nrow,Ncol)
    
    rho = 0.34  # Capillary resting net oxygen extraction (dimensionless), E_0 in Friston2000
    alpha = 0.32  # Grubb's vessel stiffness exponent (dimensionless), \alpha in Friston2000
    V0 = 0.02  # Resting blood volume fraction (dimensionless)
    k1 = 7 * rho  # (dimensionless)
    k2 = 2.0  # (dimensionless)
    k3 = 2 * rho - 0.2  # (dimensionless)
    Gamma = 0.41 * ones(Nrow)  # Rate constant for autoregulatory feedback by blood flow (1/s)
    K = 0.65 * ones(Nrow) # Vasodilatory signal decay (1/s)
    Tau = 0.98 * ones(Nrow)  # Transit time  (s)

    # initialize state variables
#     Tip: x == nothing then indicates the absence of a value,
#     and x == Some(nothing) indicates the presence of a value equal to nothing.
    if X == Some(nothing)
        X = zeros(Nrow)  # Vasso dilatory signal
    end
    if F == Some(nothing)
        F = zeros(Nrow)   # Blood flow
    end
    if Q == Some(nothing)
        Q = zeros(Nrow)   # Deoxyhemoglobin
    end
    if V == Some(nothing)
        V = zeros(Nrow)   # Blood volume
    end
    
    BOLD = zeros(Nrow,Ncol)
    
    BOLD, X, F, Q, V = integrateBOLD(BOLD, X, Q, F, V, Z, dt, N, rho, alpha, V0, k1, k2, k3, Gamma, K, Tau)
    return BOLD, X, F, Q, V
    
end

function integrateBOLD(BOLD, X, Q, F, V, Z, dt, N, rho, alpha, V0, k1, k2, k3, Gamma, K, Tau)
#     Hopefully no numba needed. We'll see TODO
    EPS = 1e-120  # epsilon for softening
    
    for i in 1:Ncol
        for j in range 1:Nrow
            X[j] = X[j] + dt * (Z[j, i] - K[j] * X[j] - Gamma[j] * (F[j] - 1))
            Q[j] = Q[j] + dt / Tau[j] * (F[j] / rho * (1 - (1 - rho) ^ (1 / F[j])) - Q[j] * V[j] ^ (1 / alpha - 1))
            V[j] = V[j] + dt / Tau[j] * (F[j] - V[j] ^ (1 / alpha))
            F[j] = F[j] + dt * X[j]
            
            F[j] = max(F[j], EPS)

            BOLD[j, i] = V0 * (k1 * (1 - Q[j]) + k2 * (1 - Q[j] / V[j]) + k3 * (1 - V[j]))
        end
    end
    return BOLD, X, F, Q, V
end

        

    
    




integrateBOLD (generic function with 1 method)